<a href="https://colab.research.google.com/github/AnttonLA/BINP37/blob/master/bioBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a small guide I wanted to write about the setup process and use of the BioBERT software (https://github.com/dmis-lab/biobert). I tried to be thorough, but I'm sure some thing are still confusing.

Please keep in mind that I am not too familiar with BioBERT myself. The things explained here I learned mostly reading comments on the GitHub page of BioBERT or through experimentaion. It is quite likely that some the things I say here are incorrect!!

#BioBERT setup in Colab (or elsewhere) for Name Entity Recognition

First step is to download all the necesary files to make bioBERT work. That includes the pre-trained weights, the required packages and the datasets used in the fine-tuning process of the model.

We'll first clone bioBERT from GitHub and make sure we have the required packages installed. There is a compatibility issue between Tensorflow (which bioBERT is based on) and the 'cuda' repository. Aparently TensorFlow is only compatible with versions 9.0 or lower. This means we will have to install an older version. **Beware, as the cell will prompt a Yes/No question that needs to be answered for the installation to procede**. The package will take some minutes to install.

In [0]:
!git clone https://github.com/dmis-lab/biobert.git

Cloning into 'biobert'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 293 (delta 7), reused 0 (delta 0), pack-reused 278
Receiving objects: 100% (293/293), 489.78 KiB | 1.57 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [0]:
!pip install -r biobert/requirements.txt

In [0]:
!sudo dpkg -i drive/My\ Drive/project_biobert/cuda-repo-ubuntu1704_9.0.176-1_amd64.deb
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda-9-0 #REMEMBER YOU HAVE TO ANSWER THE PROMPT!

Selecting previously unselected package cuda-repo-ubuntu1704.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../cuda-repo-ubuntu1704_9.0.176-1_amd64.deb ...
Unpacking cuda-repo-ubuntu1704 (9.0.176-1) ...
Setting up cuda-repo-ubuntu1704 (9.0.176-1) ...

Configuration file '/etc/apt/sources.list.d/cuda.list'
 ==> File on system created by you or by a script.
 ==> File also in package provided by package maintainer.
   What would you like to do about it ?  Your options are:
    Y or I  : install the package maintainer's version
    N or O  : keep your currently-installed version
      D     : show the differences between the versions
      Z     : start a shell to examine the situation
 The default action is to keep your current version.
*** cuda.list (Y/I/N/O/D/Z) [default=N] ? y
Installing new version of config file /etc/apt/sources.list.d/cuda.list ...
Executing: /tmp/apt-key-gpghome.c0Ff9XyVzs/gpg.1.sh --fetch-keys http://developer.downlo

Next, we will download the pre-trained weights from https://github.com/naver/biobert-pretrained

In my particular case, I decided to use the BioBERT-Base v1.1 (+ PubMed 1M) weights, which I stored in my Drive. I connected the Drive to the colab notebook and loaded the files from there.

In [0]:
!tar -xvf drive/My\ Drive/project_biobert/biobert_v1.1_pubmed.tar

biobert_v1.1_pubmed/
biobert_v1.1_pubmed/model.ckpt-1000000.data-00000-of-00001
biobert_v1.1_pubmed/model.ckpt-1000000.meta
biobert_v1.1_pubmed/bert_config.json
biobert_v1.1_pubmed/vocab.txt
biobert_v1.1_pubmed/model.ckpt-1000000.index


The main bioBERT GitHub page offers eight datasets to perfrom the fine-tuning with. These datasets can be found here: https://github.com/dmis-lab/biobert 

Again, I had the files stored in my Drive and loaded them from there.

In [0]:
!unzip drive/My\ Drive/project_biobert/NERdata.zip

Archive:  drive/My Drive/project_biobert/NERdata.zip
   creating: BC2GM/
  inflating: BC2GM/devel.tsv         
  inflating: BC2GM/test.tsv          
  inflating: BC2GM/train.tsv         
  inflating: BC2GM/train_dev.tsv     
   creating: BC4CHEMD/
  inflating: BC4CHEMD/devel.tsv      
  inflating: BC4CHEMD/test.tsv       
  inflating: BC4CHEMD/train.tsv      
  inflating: BC4CHEMD/train_dev.tsv  
   creating: BC5CDR-chem/
  inflating: BC5CDR-chem/devel.tsv   
  inflating: BC5CDR-chem/test.tsv    
  inflating: BC5CDR-chem/train.tsv   
  inflating: BC5CDR-chem/train_dev.tsv  
   creating: BC5CDR-disease/
  inflating: BC5CDR-disease/devel.tsv  
  inflating: BC5CDR-disease/test.tsv  
  inflating: BC5CDR-disease/train.tsv  
  inflating: BC5CDR-disease/train_dev.tsv  
   creating: JNLPBA/
  inflating: JNLPBA/devel.tsv        
  inflating: JNLPBA/test.tsv         
  inflating: JNLPBA/train.tsv        
  inflating: JNLPBA/train_dev.tsv    
   creating: linnaeus/
  inflating: linnaeus/devel.tsv

With this, we have all of the scripts, packages and data we will need loaded on Colab. Before we procede with the fine tuning, we will create some environmental variables to make our lives easier.

The variables will contain the paths to the pre-trained model, the datasets, and the location where the output files should be stored. We will also create this output folder.

In [0]:
import os
os.environ['BIOBERT_DIR']= './biobert_v1.1_pubmed' #Pre-trained model.
os.environ['NER_DIR'] = './BC5CDR-chem' # Dataset. Could be any dataset you want to use.
os.environ['OUTPUT_DIR'] = './ner_outputs' # Output
!mkdir $OUTPUT_DIR

# Fine tunning

Everyting is ready for the actual fine-tuning. **Make sure to change the Colab notebook to GPU mode**, otherwise it will take too long to execute. You can do this by going to 'Edit > Configure Notebook'.

Check how much GPU you have access to, as the training will fail if you don't have enough. In my experience, you want to have at least around 15000MB

In [0]:
#CHECK HOW MUCH GPU COLAB IS GIVING YOU
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()


  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=522afaa4377a4e2e1e81c276de5033905459bc6cee9f2cea274cc092b74fc044
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 158.8 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


The training will take a while to complete, depending on the number of epochs selected and the sentence length.

The GitHub page has more information about the specific flags of the *run_ner.py* script. Otherwise you can use '--help'.

In [0]:
!python biobert/run_ner.py --do_train=true --do_eval=true --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --num_train_epochs=10.0 --data_dir=$NER_DIR --output_dir=$OUTPUT_DIR

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

Once the model has been fine-tuned to the dataset we have selected, we will get (among other things) two .txt files in our output directory: **token_test.txt** and **label_test.txt**. These are the main output files of the *run_ner.py* script other than the model itself.

They can be used to asses the accuracy of the model against the default test-set (that is, the file 'test.tsv' in whichever folder we have selected as $NER_DIR). In order to get this data, we will need to "detokenize" the output files. We will use a script in biobert called _ner_detokenize.py_ to do so. This will produce a .txt file named **NER_result_conll.txt** in our output directory. This file contains three columns of text, the first being a word, the second being the "true" classification, and the third being the classification made by our model.

The three possible classifications are: **B-MISC** (Beggining of Entity), **I-MISC** (Inside Entity) or **O-MISC** (Outside Entity).

The Perl script *conlleval.pl* gives the proper statistics of the resutls.

**NOTE regarding sentence length and _ner_detokenize.py_:** if a sentence is longer than the sentence-length specified when running _run_ner.py_, the tokens/words located past the length limit will not be evaluated, and will automatically recieve an **O-MISC** tag. This will later produce a warning when running _ner_detokenize.py_.


In [0]:
#Detokenization
!python biobert/biocodes/ner_detokenize.py --token_test_path=$OUTPUT_DIR/token_test.txt --label_test_path=$OUTPUT_DIR/label_test.txt --answer_path=$NER_DIR/test.tsv --output_dir=$OUTPUT_DIR

In [0]:
#Take a look at the output file.
!head $OUTPUT_DIR/NER_result_conll.txt 

Torsade O-MISC O-MISC
de O-MISC O-MISC
pointes O-MISC O-MISC
ventricular O-MISC O-MISC
tachycardia O-MISC O-MISC
during O-MISC O-MISC
low O-MISC O-MISC
dose O-MISC O-MISC
intermittent O-MISC O-MISC
dobutamine B-MISC B-MISC


In [0]:
#Evaluation of the results
!perl biobert/biocodes/conlleval.pl < $OUTPUT_DIR/NER_result_conll.txt

processed 124750 tokens with 5385 phrases; found: 5401 phrases; correct: 5043.
accuracy:  99.22%; precision:  93.37%; recall:  93.65%; FB1:  93.51
             MISC: precision:  93.37%; recall:  93.65%; FB1:  93.51  5401


# Using our model for predition

Once we know that our model is working as intended and is capable of classifying the entities we are interested in, we want to put it to work with our own data.

However, while BioBERT is certainly capable of doing this, there are certain limitations to using it for predicition with our own data, mainly related to data formatting. I'll try to explain the problems (and their solutions) as well as possible. 

While the GitHub page claims that the *run_ner.py* script can be used in "classifier mode" by altering the flags *---do_train=false* and *---do_predict=true*, this does not work properly. Instead, we will need to use a little workaround.

We will keep *---do_train==true*. We will, however, decrease the number of epochs. This way, since out fine-tuned model is in the directory *ner_outputs/* but the epochs are fewer than before, the software will skip the training process, and will insetead perform the prediction directly.

Unfortunately, we will still need to have the training datasets for this to work. Therefore, our data will need to be in a file called **test.tsv**, and be stored in the path saved at $NER_DIR along with the files *devel.tsv*, *train_dev.tsv* and *train.tsv*. The easiest way of doing this is taking one of the datasets for the fine-tunning process and swapping the **test.tsv** file for our data. I have written some code that performs this formatting, asuming the data you want to classify is in a specific JSON format. This code can be found below in the section "Creating our own 'test.tsv' files".

Wether you use this code or do it yourself from scratch, **beware of the format of the test.tsv file!** Every line will need to be composed of *word* + *\\t* + *label* + *\\n*. The label is a single character, either an "O", an "I" or a "B". In the original test-sets, it corresponds to the true evaluation of the word before it. In our case, it doesn't matter at all, since we are producing this file just to make BioBERT work. I recommend setting an "O" to every word for simplicity. **The last line of the file also needs to be a newline (\\n) character**, otherwise the script won't work.

In [0]:
#Prediction
!python biobert/run_ner.py --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --data_dir=$NER_DIR --do_train=true --num_train_epochs=1.0 --do_predict=true --output_dir=$OUTPUT_DIR

Detokenize the output files again, same as in the training. 

In [0]:
!python biobert/biocodes/ner_detokenize.py --token_test_path=$OUTPUT_DIR/token_test.txt --label_test_path=$OUTPUT_DIR/label_test.txt --answer_path=$NER_DIR/test.tsv --output_dir=$OUTPUT_DIR

The classification made by BioBERT will be stored in _ner_outputs/NER_result_conll.txt_. This time, we can ignore the second colum, since it countains our 'fake' labels. The third column of the file is the classification made by our model. 

Extracting the information from this file can be a little tricky. I've written a solution that can be found below in the section "Extracting output informatio". Hope this helps!

**Congratulations! If you reached this far you have succesfully used BioBERT for classification.**

#Creating our own 'test.tsv' files

I want to transform the given 100-paper dataset into a 'test.tsv' file in order to perform NER in those papers.

In order to accomplsh this, we will need to extract all the text from the dataset, and reformat it so that each word and special character is in a different line, with each line containing a fake "tag".

The files I used for this can be found here: https://github.com/AnttonLA/BINP37/tree/master/dataset_generation

First, the script _json_to_txt.py_ will extract all the text from the JSON files. It needs the path to where the files are located (in the script, under the variable name _mypath_) and the metadata file that comes with the dataset. 

It will output two files: _generated_text.txt_ (the full text of all of the papers in the dataset in a single .txt file) and 'rebuild_reference.txt'. This second file isn't important right now. It is used later to reformat the output of BioBERT (_NER_result_conll.txt_) to PubAnnotation format. More on this on the next section.

After we obtain _generated_text.txt_, the script 'text_to_tsv.py' will take the file and transform it to the same format as _test.tsv_. The final file will be named **_generated_test.tsv_**. This file can be then used to replace the _test.tsv_ file in the $NER_DIR dataset. Once the replacement is done and _generated_test.tsv_ is renamed to simply _test.tsv_, we can run the classification using BioBERT. 

# Extracting output information

Once we have obtained our output file (_NER_result_conll.txt_) we want to reformat our output so that it is on the Pubannotation format. 

I have written some code in order to do this. It can be found here: https://github.com/AnttonLA/BINP37/tree/master/output_generation

The script _eval_to_pubanot.py_ takes the files _NER_result_conll.txt_ and _rebuild_reference.txt_ (generated, as explained in the previous section, by _json_to_txt.py_ during the fake test-set generation). It uses these to files to generate a JSON file per each paragraph or section of text in our input, noting those words that have been identified by BioBERT as entities.